In [1]:
import pandas as pd
import numpy as np
from babel.numbers import format_currency

In [2]:
index_name = "P0P4"
etf_name = "PGX"

# Output File Name
outputFile = 'Output/{} ETF vs ICE.xlsx'.format(etf_name)

# Total market Cap for PFF, needs to be manually updated
# Can be automated later
current_pgx_market_cap = 4902900000
current_pgx_cash = 40000000

# Reading ICE Data For Projected Universe
projected_universe_df = pd.read_excel('Data/P0P4-Projected.xlsx', skiprows=[0])

# Reading PFF Website Data for Current Shares
pgx_current_holdings_df = pd.read_csv('Data/PGX_Website_Data.csv')
pgx_current_holdings_df = pgx_current_holdings_df[[' Holding Ticker', ' Shares/Par Value']]
pgx_current_holdings_df[' Shares/Par Value'] = pgx_current_holdings_df[' Shares/Par Value'].str.replace(',','').astype(int) 
# DataFrame for our final Output
pgx_df = pd.DataFrame()


# When reading data, below strings are read, they must be marked as np.nan
msg1 = 'Any unauthorized use or disclosure is prohibited. Nothing herein should in any way be deemed to alter the legal rights and obligations contained in agreements between any ICE Data Services entity ("ICE") and their clients relating to any of the Indices or products or services described herein. The information provided by ICE and contained herein is subject to change without notice and does not constitute any form of representation, or undertaking.  ICE and its affiliates make no warranties whatsoever, either express or implied, as to merchantability, fitness for a particular purpose, or any other matter in connection with the information provided. Without limiting the foregoing, ICE and its affiliates makes no representation or warranty that any information provided hereunder are complete or free from errors, omissions, or defects. All information provided by ICE is owned by or licensed to ICE. ICE retains exclusive ownership of the ICE Indices, including the ICE BofAML Indexes, and the analytics used to create this analysis ICE may in its absolute discretion and without prior notice revise or terminate the ICE information and analytics at any time. The information in this analysis is for internal use only and redistribution of this information to third parties is expressly prohibited.'
msg2 = 'Neither the analysis nor the information contained therein constitutes investment advice or an offer  or an invitation to make an offer  to buy or sell any securities or any options  futures or other derivatives related to such securities. The information and calculations contained in this analysis have been obtained from a variety of sources  including those other than ICE and ICE does not guarantee their accuracy.  Prior to relying on any ICE information and/or the execution of a security trade based upon such ICE information, you are advised to consult with your broker or other financial representative to verify pricing information. There is no assurance that hypothetical results will be equal to actual performance under any market conditions. THE ICE INFORMATION IS PROVIDED TO THE USERS "AS IS." NEITHER ICE, NOR ITS AFFILIATES, NOR ANY THIRD PARTY DATA PROVIDER WILL BE LIABLE TO ANY USER OR ANYONE ELSE FOR ANY INTERRUPTION, INACCURACY, ERROR OR OMISSION, REGARDLESS OF CAUSE, IN THE ICE INFORMATION OR FOR ANY DAMAGES RESULTING THEREFROM. In no event shall ICE or any of its affiliates, employees  officers  directors or agents of any such persons have any liability to any person or entity relating to or arising out of this information, analysis  or the indices  contained herein.'

# Projected Universe Data
projected_universe_df = projected_universe_df.replace('NaN', np.nan)
projected_universe_df = projected_universe_df.replace(msg1, np.nan)
projected_universe_df= projected_universe_df.replace(msg2, np.nan)

#drop row if ISIN number is Nan
projected_universe_df.dropna(subset=['ISIN number'], inplace=True)

In [3]:
pgx_website_data_corrections = {
    'ATH-PA': 'ATH.PRA',
    'SCE-PK': 'SCE.PRK',
    'PSA-PM': 'PSA.PRM',
    'CTA-PB': 'CTA.PRB',
    'TRTN-PB': 'TRTN.PRB',
    'SJI': 'SJIJ'
}

# Changing the Ticker in PGX Website Data to IB Format
def getIBFormat(x):
    return ".PR".join(x.split(' '))

pgx_current_holdings_df['Ticker']=pgx_current_holdings_df[' Holding Ticker'].apply(getIBFormat)

## Applying Corrections in the Ticker format
for holding_ticker in pgx_website_data_corrections:
    pgx_current_holdings_df.loc[pgx_current_holdings_df[' Holding Ticker']==holding_ticker, 'Ticker'] = pgx_website_data_corrections[holding_ticker]

In [4]:
isin_to_ticker_df = pd.read_excel('Static Data/ISINtoTicker.xlsx')

In [5]:
pgx_current_holdings_df = pgx_current_holdings_df.merge(isin_to_ticker_df, left_on='Ticker', right_on='Ticker', how='left')

In [6]:
# Storing the unique ISIN ID's from both the current and projected universe data
unique_ISIN = pd.concat([pgx_current_holdings_df['ISIN'], projected_universe_df['ISIN number']]).drop_duplicates().reset_index(drop=True)
pgx_df['ISIN number'] = unique_ISIN

In [7]:
pgx_df = pgx_df.merge(isin_to_ticker_df, left_on="ISIN number", right_on="ISIN", how="left").drop(["ISIN"], axis=1)

In [8]:
def getProjectedMarketCap(x):
    isid = x['ISIN number']
    ticker = projected_universe_df[projected_universe_df['ISIN number']==isid]
    if(len(ticker)>0):
        return ticker.iloc[0]['% Mkt Value']
    return 0

pgx_df['Projected % Mkt Cap'] = pgx_df.apply(getProjectedMarketCap, axis=1)

In [9]:
def getCurrentPGXShares(x):
    isin = x['ISIN number']
    ticker = pgx_current_holdings_df[pgx_current_holdings_df['ISIN']==isin]
    if(len(ticker)>0):
        return ticker.iloc[0][' Shares/Par Value']
    return 0

pgx_df['Current PGX Shares'] = pgx_df.apply(getCurrentPGXShares, axis=1)

In [10]:
pgx_df.drop(['Activ Ticker', 'Bloomberg File Ticker', 'Activ Last Price Formula'], axis=1, inplace=True)

In [11]:
def getProjectedPGXShares(x):
    if(not isinstance(x['Last Price'], str)):
        return np.rint((current_pgx_market_cap*x['Projected % Mkt Cap'])/(x['Last Price']*100))
    return 0

pgx_df['Projected PGX Shares'] = pgx_df.apply(getProjectedPGXShares, axis=1)

In [12]:
def getDifference(x):
    if((not isinstance(x['Projected PGX Shares'], str)) and (not isinstance(x['Current PGX Shares'], str))):
        return x['Projected PGX Shares']-x['Current PGX Shares']
    return np.nan

def getDollarDifference(x):
    if((not isinstance(x['Last Price'], str)) and (not isinstance(x['Difference'], str))):
        return x['Last Price']*x['Difference']
    return np.nan

pgx_df['Difference'] = pgx_df.apply(getDifference, axis=1)
pgx_df['Difference $'] = pgx_df.apply(getDollarDifference, axis=1)

In [13]:
pgx_df.sort_values('Difference')
pgx_df.sort_values(by='Difference $', key=abs, ascending=False, inplace=True)
pgx_df['Difference $'] = pgx_df['Difference $'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))


In [14]:
# Number of Shares
total_buys_ice = pgx_df['Difference'].where(pgx_df['Difference']>0).sum()
total_sells_ice = pgx_df['Difference'].where(pgx_df['Difference']<0).sum()

# Dollar Value
# total_dollar_buying = pgx_df[pgx_df['Difference']>0].apply(lambda x: x['Last Price']*x['Difference'], axis=1).sum()
# total_dollar_selling = pgx_df[pgx_df['Difference']<0].apply(lambda x: x['Last Price']*x['Difference'], axis=1).sum()

total_pgx_transactions_df = pd.DataFrame({"Number of Shares":[total_buys_ice, 
                                                              total_sells_ice,
#                                                               total_dollar_buying,
#                                                               abs(total_dollar_selling),
                                                              current_pgx_market_cap,
                                                              current_pgx_cash
                                                             ]}, 
                                         index=['Total PGX Buys (Number of Shares)', 
                                                'Total PGX Sells (Number of Shares)',
#                                                 'Total PGX Buys (in $)',
#                                                 'Total PGX Sells (in $)',
                                                'PGX NAV',
                                                'PGX Cash'
                                               ])

In [15]:
with pd.ExcelWriter(outputFile, mode="w") as writer:
    pgx_df.to_excel(writer, sheet_name='PGX')
    total_pgx_transactions_df.to_excel(writer, sheet_name='Total Transactions - PGX')
    
    workbook = writer.book
    worksheet = writer.sheets['PGX']

    cellFormat = workbook.add_format({'num_format': '#,###'})
    worksheet.set_column('F:H', 10, cellFormat)